In [1]:
import os
import re
import pandas as pd 
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
non_ascii_replacement_dict = {
    '€': '<euro>',
    '¢': '<cent>',
    '₹': '<rupee>',
    '×': 'X',
    '²': '<pow2>',
    '³': '<pow3>',
    '™': '<trademark>',
    '®': '<registered>',
    '–': '<dash>'
}

In [3]:
# Define util functions to do cleanup and pattern matching

def clean_text(text):
    if isinstance(text, str):
        text = text.replace('“', '"').replace('”', '"').replace('″', '"')
        text = text.replace('‘', "'").replace('’', "'")

        # replace space and 2 or more double quotes with one & remove space 
        text = re.sub(r'(\d)\s?""*', r'\1"', text)
        
        text = re.sub(r"\s?''", r'"', text)
       
        # replace degree sign with text
        text = re.sub(r'\s?°', r'<deg>', text)

        # replace the non-ascii characters in the dict with their defined replacement         
        for pattern, replace in non_ascii_replacement_dict.items():
            text = re.sub(re.escape(pattern), re.escape(replace), text)

        text = re.sub(r'[^\x00-\x7F]', '', text)    
    return text


def to_uppercase(text):
    if isinstance(text, str):
        text = text.upper()
    return text


def clean_item_code(value):
    # Pattern for 2 letters followed by 5 digits -> ^[A-Za-z]{2}\d{5}$

    value = re.sub(r'^0+', '', value)
    clean_value = value
    
    pattern = r'^(\d{5})(0*)$'
    match = re.search(pattern, value)

    if match:
        clean_value = match.group(1)

    return clean_value

### Load and Clean Purchase Order Line Items Dataset

In [4]:
df = pd.read_excel(os.path.join(os.getenv('PURCHASE_ORDER_FOLDER_PATH'), 'Data 1.xlsx'))
df.columns = df.columns.str.replace(' ', '_').str.upper()

In [5]:
df

,#,DOCTYPE,CANCELED,DOCCUR,DOCRATE,DOCSTATUS,PO_NUM,BASE_TYPE,BASE_NUMBER,BASE_LINE,...,MANUALLY_CLOSED_PO,PO_OWNER_NAME,PYMNTGROUP,HEADER,FOOTER,BRANCH_NAME,IMP_OR_EXP,COST_SAVING,COST_SAVING_CRITERIA,FORCE_CLOSED_PO_QTY
0,26968,I,N,INR,1.0,C,232431279,PR,232401368.0,1.0,...,No,Yoginder Kumar,Net 30 days,NaN,NaN,NOIDA,N,NaN,NaN,0.0
1,42455,I,N,INR,1.0,C,232471206,PR,232451256.0,1.0,...,No,Ashutosh Tiwari,Net 30 days,NaN,NaN,Ecotech X,N,NaN,NaN,0.0
2,45582,S,N,INR,1.0,C,232472169,PR,232452086.0,1.0,...,No,Abhishek Chauhan,Refer PO Text,Payment Terms : 100% after service completion ...,NaN,Ecotech X,N,NaN,NaN,0.0
3,9868,S,N,INR,1.0,C,72616,PR,100150.0,1.0,...,Yes,Aman Chambial,Net-30,OTC charges shall be paid after installation o...,NaN,NOIDA,N,NaN,NaN,0.0
4,9869,S,N,INR,1.0,C,72616,PR,100150.0,2.0,...,Yes,Aman Chambial,Net-30,OTC charges shall be paid after installation o...,NaN,NOIDA,N,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50456,48133,I,N,INR,1.0,O,242504101,PR,242500075.0,6.0,...,No,Dheeraj Sachdeva,Net 30 days,NaN,NaN,NOIDA,N,NaN,NaN,0.0
50457,48134,I,N,INR,1.0,O,242504101,PR,242500075.0,4.0,...,No,Dheeraj Sachdeva,Net 30 days,NaN,NaN,NOIDA,N,NaN,NaN,0.0
50458,48135,I,N,INR,1.0,O,242504101,PR,242500075.0,3.0,...,No,Dheeraj Sachdeva,Net 30 days,NaN,NaN,NOIDA,N,NaN,NaN,0.0
50459,16606,I,N,INR,1.0,C,74346,PR,102315.0,1.0,...,No,Sanket Singh Rathor,100% advance against PI,12 mtr weight is 948 kg (200*200) 140*90 it i...,NaN,NOIDA,N,NaN,NaN,0.0


In [6]:
df_cleaned = df.apply(lambda col: col.apply(clean_text))
df_cleaned = df_cleaned.apply(lambda col: col.apply(to_uppercase))

In [35]:
df_cleaned[df_cleaned['PO_VALUE'] == 0]

,#,DOCTYPE,CANCELED,DOCCUR,DOCRATE,DOCSTATUS,PO_NUM,BASE_TYPE,BASE_NUMBER,BASE_LINE,...,MANUALLY_CLOSED_PO,PO_OWNER_NAME,PYMNTGROUP,HEADER,FOOTER,BRANCH_NAME,IMP_OR_EXP,COST_SAVING,COST_SAVING_CRITERIA,FORCE_CLOSED_PO_QTY
6416,31589,I,N,INR,1.0,C,232432821,PR,232403244.0,3.0,...,NO,KESHAV CHAUHAN,REFER PO TEXT,NaN,NaN,NOIDA,N,NaN,NaN,0.0
6417,31590,I,N,INR,1.0,C,232432821,PR,232403244.0,2.0,...,NO,KESHAV CHAUHAN,REFER PO TEXT,NaN,NaN,NOIDA,N,NaN,NaN,0.0
6418,31591,I,N,INR,1.0,C,232432821,PR,232403244.0,1.0,...,NO,KESHAV CHAUHAN,REFER PO TEXT,NaN,NaN,NOIDA,N,NaN,NaN,0.0
6419,31592,I,N,INR,1.0,C,232432821,PR,232403244.0,5.0,...,NO,KESHAV CHAUHAN,REFER PO TEXT,NaN,NaN,NOIDA,N,NaN,NaN,0.0
6420,31593,I,N,INR,1.0,C,232432821,PR,232403244.0,6.0,...,NO,KESHAV CHAUHAN,REFER PO TEXT,NaN,NaN,NOIDA,N,NaN,NaN,0.0
6421,31594,I,N,INR,1.0,C,232432821,PR,232403244.0,7.0,...,NO,KESHAV CHAUHAN,REFER PO TEXT,NaN,NaN,NOIDA,N,NaN,NaN,0.0
6422,31595,I,N,INR,1.0,C,232432821,PR,232403244.0,8.0,...,NO,KESHAV CHAUHAN,REFER PO TEXT,NaN,NaN,NOIDA,N,NaN,NaN,0.0
6423,31596,I,N,INR,1.0,C,232432821,PR,232403244.0,9.0,...,NO,KESHAV CHAUHAN,REFER PO TEXT,NaN,NaN,NOIDA,N,NaN,NaN,0.0
6424,31597,I,N,INR,1.0,C,232432821,PR,232403244.0,10.0,...,NO,KESHAV CHAUHAN,REFER PO TEXT,NaN,NaN,NOIDA,N,NaN,NaN,0.0
6425,31598,I,N,INR,1.0,C,232432821,PR,232403244.0,11.0,...,NO,KESHAV CHAUHAN,REFER PO TEXT,NaN,NaN,NOIDA,N,NaN,NaN,0.0


In [7]:
df_cleaned_non_nan = df_cleaned[df_cleaned['ITEM_CODE'].notna()].copy()
df_cleaned_non_nan

,#,DOCTYPE,CANCELED,DOCCUR,DOCRATE,DOCSTATUS,PO_NUM,BASE_TYPE,BASE_NUMBER,BASE_LINE,...,MANUALLY_CLOSED_PO,PO_OWNER_NAME,PYMNTGROUP,HEADER,FOOTER,BRANCH_NAME,IMP_OR_EXP,COST_SAVING,COST_SAVING_CRITERIA,FORCE_CLOSED_PO_QTY
0,26968,I,N,INR,1.0,C,232431279,PR,232401368.0,1.0,...,NO,YOGINDER KUMAR,NET 30 DAYS,NaN,NaN,NOIDA,N,NaN,NaN,0.0
1,42455,I,N,INR,1.0,C,232471206,PR,232451256.0,1.0,...,NO,ASHUTOSH TIWARI,NET 30 DAYS,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0
11,28046,I,Y,INR,1.0,C,232431518,PR,232401440.0,1.0,...,NO,AMAN CHAMBIAL,NET 30 DAYS,NaN,NaN,NOIDA,N,NaN,NaN,0.0
12,28047,I,Y,INR,1.0,C,232431518,PR,232401440.0,2.0,...,NO,AMAN CHAMBIAL,NET 30 DAYS,NaN,NaN,NOIDA,N,NaN,NaN,0.0
13,21664,I,N,INR,1.0,C,75796,PR,103757.0,3.0,...,NO,ABHISHEK CHAUHAN,100 % AGAINST MATERIAL DELIVERY,NaN,NaN,NOIDA,N,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50456,48133,I,N,INR,1.0,O,242504101,PR,242500075.0,6.0,...,NO,DHEERAJ SACHDEVA,NET 30 DAYS,NaN,NaN,NOIDA,N,NaN,NaN,0.0
50457,48134,I,N,INR,1.0,O,242504101,PR,242500075.0,4.0,...,NO,DHEERAJ SACHDEVA,NET 30 DAYS,NaN,NaN,NOIDA,N,NaN,NaN,0.0
50458,48135,I,N,INR,1.0,O,242504101,PR,242500075.0,3.0,...,NO,DHEERAJ SACHDEVA,NET 30 DAYS,NaN,NaN,NOIDA,N,NaN,NaN,0.0
50459,16606,I,N,INR,1.0,C,74346,PR,102315.0,1.0,...,NO,SANKET SINGH RATHOR,100% ADVANCE AGAINST PI,12 MTR WEIGHT IS 948 KG (200*200) 140*90 IT I...,NaN,NOIDA,N,NaN,NaN,0.0


In [8]:
df_cleaned_non_nan['ITEM_CODE_CLEANED'] = df_cleaned_non_nan['ITEM_CODE'].astype(str).apply(clean_item_code)
df_cleaned_non_nan[['ITEM_CODE', 'ITEM_CODE_CLEANED']]

,ITEM_CODE,ITEM_CODE_CLEANED
0,MC11560,MC11560
1,4556800,45568
11,ST00545,ST00545
12,ST00546,ST00546
13,GC01436,GC01436
...,...,...
50456,00047513A,47513A
50457,00047494A,47494A
50458,00047510A,47510A
50459,3079000,30790


In [9]:
def keep_most_quantity(group):
    return group.loc[group['ORDERED_QUANTITY'].idxmax()]

In [10]:
non_nan_filtered = df_cleaned_non_nan.groupby(['PO_NUM', 'ITEM_CODE_CLEANED']).apply(keep_most_quantity).reset_index(drop=True)
non_nan_filtered

/var/folders/4c/99wh1f85111309s8ydsfjmlr0000gn/T/ipykernel_49860/2495202280.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  non_nan_filtered = df_cleaned_non_nan.groupby(['PO_NUM', 'ITEM_CODE_CLEANED']).apply(keep_most_quantity).reset_index(drop=True)


,#,DOCTYPE,CANCELED,DOCCUR,DOCRATE,DOCSTATUS,PO_NUM,BASE_TYPE,BASE_NUMBER,BASE_LINE,...,PO_OWNER_NAME,PYMNTGROUP,HEADER,FOOTER,BRANCH_NAME,IMP_OR_EXP,COST_SAVING,COST_SAVING_CRITERIA,FORCE_CLOSED_PO_QTY,ITEM_CODE_CLEANED
0,4,I,N,INR,1.0,C,51038,-1,NaN,NaN,...,NaN,NET 15 DAYS,NaN,NaN,NOIDA,N,NaN,NaN,140000.0,ELC10849
1,5,I,N,INR,1.0,C,51038,-1,NaN,NaN,...,NaN,NET 15 DAYS,NaN,NaN,NOIDA,N,NaN,NaN,100000.0,ELC10884
2,9,I,Y,INR,1.0,C,53586,-1,NaN,NaN,...,NaN,NET 15 DAYS,PLEASE SEE THE BELOW TENTATIVE DELIVERY TIMELI...,NaN,NOIDA,N,NaN,NaN,0.0,17340
3,10,I,Y,INR,1.0,C,53586,-1,NaN,NaN,...,NaN,NET 15 DAYS,PLEASE SEE THE BELOW TENTATIVE DELIVERY TIMELI...,NaN,NOIDA,N,NaN,NaN,0.0,EL14328
4,12,I,N,INR,1.0,C,53650,-1,NaN,NaN,...,NaN,100 % AGAINST MATERIAL DELIVERY,NaN,NaN,NOIDA,N,NaN,NaN,1.0,EL14297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42154,50454,I,N,INR,1.0,O,242530645,PR,242550434.0,1.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,16132
42155,50455,I,N,INR,1.0,O,242530645,PR,242550443.0,20.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,16135
42156,50458,I,N,INR,1.0,O,242530645,PR,242550443.0,4.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,175
42157,50460,I,N,INR,1.0,O,242530645,PR,242550420.0,2.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,22952


### Load and Clean Electrical Parts Catalogue Dataset

In [11]:
electrical_parts = pd.read_csv(
    os.path.join(os.getenv('ELECTRICAL_PARTS_FOLDER_PATH'), 'Electrical Parts Report Modified.csv'),
)
electrical_parts.columns = electrical_parts.columns.str.replace(' ', '_').str.upper()

In [13]:
electrical_parts

,PART_ID,PART_NAME,PART_DESCRIPTION,PART_OWNER,PART_REVISION,REVISION_DATE,PRODUCT_GROUP,PRODUCT_SUBGROUP,WIDTH_(MM),HEIGHT_(MM),...,MOUNTING_CLEARANCES_(MM),WEIGHT_(KG),MANUFACTURER,MANUFACTURER_PART_NUMBER,EPLAN_PART_NUMBER,OLD_PLM_ID,OLD_SAP_ID,RELEASED_STATUS,RELEASED_DATE,PART_CATEGORY
0,41237,"CABLE GLAND- DOUBLE COMPRESSION CABLE, M90","CABLE GLAND DOUBLE COMPRESSION, SS304, OD:72-8...",Kumari Monika (kumarimonika),0,31-Aug-2023 16:47,0.0,0.0,0.0,0.0,...,NaN,0.000,CGS,CGW 014,CGS.CGW 014,NaN,NaN,Production,18-Sep-2023 10:46,Electrical Part
1,52897,"E-STOP- LED, SELF MONTORING, 2CH",EMERGENCY PUSH BUTTON,Kumari Monika (kumarimonika),0,01-Mar-2024 16:47,12.0,1.0,0.0,0.0,...,NaN,2.700,ROCKWELL,800F-1YMD81,A-B.800F-1YMD81,NaN,NaN,Production,08-Mar-2024 11:36,Electrical Part
2,54675,"INTERFACE MODULE-PLC,RS485",CPU - CENTRAL PROCESSING UNITS NX1P RS-422/485...,Kumari Monika (kumarimonika),0,11-Apr-2024 12:38,26.0,1.0,0.0,0.0,...,NaN,0.000,OMRON,NX1W-CIF11,OMR.NX1W-CIF11,NaN,NaN,Production,12-Apr-2024 10:46,Electrical Part
3,57680,"BRAKE CONNECTOR-FOR HF-SE/SN/SP/JP,HG-SN/SR,HK...","BRAKE CONNECTOR FOR HF-SE/SN/SP/JP,HG-SN/SR,HK...",Kumari Monika (kumarimonika),0,31-May-2024 12:11,129.0,1.0,90.0,60.0,...,NaN,0.050,MITSUBISHI,MR-BKCNS1-2M-L,MIT.MR-BKCNS1-2M-L,NaN,NaN,Production,31-May-2024 16:43,Electrical Part
4,64912,CONTACT-1NO,WITHOUT HOLDER - NON-ILLUMINATED - SINGLE CONT...,Kumari Monika (kumarimonika),0,21-Sep-2024 14:05,6.0,194.0,10.0,44.0,...,NaN,0.013,ABB,1SFA611610R1001,ABB.1SFA611610R1001,NaN,NaN,Production,23-Sep-2024 13:59,Electrical Part
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9354,46980,"ÖLFLEX® CLASSIC 100 5G1,5",Power and control cables / Wide-range use / PV...,Kumari Monika (kumarimonika),0,28-Nov-2023 11:09,29.0,1.0,0.0,0.0,...,NaN,0.000,LAPP,100664,LAPP.00100664,NaN,NaN,NaN,NaN,Electrical Part
9355,46984,"ÖLFLEX® CLASSIC 100 H 5G2,5",NaN,Kumari Monika (kumarimonika),0,28-Nov-2023 11:10,29.0,1.0,0.0,0.0,...,NaN,0.000,LAPP,14159,LAPP.0014159,NaN,NaN,NaN,NaN,Electrical Part
9356,46987,"ÖLFLEX® FD ROBUST 756 C 4 G 2,5+(2x1)",Power and control cables / Power chain applica...,Kumari Monika (kumarimonika),0,28-Nov-2023 11:10,29.0,1.0,0.0,0.0,...,NaN,0.000,LAPP,36061,LAPP.0036061,NaN,NaN,NaN,NaN,Electrical Part
9357,46985,"ÖLFLEX® SERVO 700 4G1,5+(2x0,75)FDF",Power and control cables / SERVO-applications ...,Kumari Monika (kumarimonika),0,28-Nov-2023 11:10,29.0,1.0,0.0,0.0,...,NaN,0.000,LAPP,36025,LAPP.0036025,NaN,NaN,NaN,NaN,Electrical Part


In [14]:
electrical_parts_cleaned = pd.DataFrame(electrical_parts.apply(lambda col: col.apply(clean_text)))
electrical_parts_cleaned = electrical_parts_cleaned.apply(lambda col: col.apply(to_uppercase))

In [15]:
electrical_parts_cleaned['PART_ID_CLEANED'] = electrical_parts_cleaned['PART_ID'].astype(str).apply(clean_item_code)
electrical_parts_cleaned[['PART_ID', 'PART_ID_CLEANED']]

,PART_ID,PART_ID_CLEANED
0,41237,41237
1,52897,52897
2,54675,54675
3,57680,57680
4,64912,64912
...,...,...
9354,46980,46980
9355,46984,46984
9356,46987,46987
9357,46985,46985


In [16]:
def keep_latest_revision(group):
    return group.loc[group['PART_REVISION'].idxmax()]

In [17]:
electrical_cleaned_filtered = electrical_parts_cleaned.groupby('PART_ID_CLEANED').apply(keep_latest_revision).reset_index(drop=True)
electrical_cleaned_filtered

/var/folders/4c/99wh1f85111309s8ydsfjmlr0000gn/T/ipykernel_49860/339672267.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  electrical_cleaned_filtered = electrical_parts_cleaned.groupby('PART_ID_CLEANED').apply(keep_latest_revision).reset_index(drop=True)


,PART_ID,PART_NAME,PART_DESCRIPTION,PART_OWNER,PART_REVISION,REVISION_DATE,PRODUCT_GROUP,PRODUCT_SUBGROUP,WIDTH_(MM),HEIGHT_(MM),...,WEIGHT_(KG),MANUFACTURER,MANUFACTURER_PART_NUMBER,EPLAN_PART_NUMBER,OLD_PLM_ID,OLD_SAP_ID,RELEASED_STATUS,RELEASED_DATE,PART_CATEGORY,PART_ID_CLEANED
0,10071,10071,NaN,HIMANSHU MEHTA (HIMANSHUMEHTA),0,16-APR-2024 15:45,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ELECTRICAL PART,10071
1,14730,MOTOR,"100W, 24VDC BLDC MOTOR, 90MM FRAME, 2500RPM,6A...",KUMARI MONIKA (KUMARIMONIKA),0,13-OCT-2022 20:20,9.0,1.0,0.0,0.0,...,0.0,ORIENTAL,BLHM5100K-GFS,ORM.BLHM5100K-GFS,NaN,NaN,PRODUCTION,18-JAN-2023 20:06,ELECTRICAL PART,14730
2,14857,"CABLE,M8 RIGHT ANGLED TO FREE LEADS,2M","SENSOR/ACTUATOR CABLE, 3-POSITION, PVC, YELLOW...",KUMARI MONIKA (KUMARIMONIKA),0,18-OCT-2022 11:35,29.0,184.0,0.0,0.0,...,0.0,PHEONIX.C,1406321,PXC.1406321,NaN,NaN,NaN,NaN,ELECTRICAL PART,14857
3,14858,CONNECTOR,"CONNECTOR, UNIVERSAL, 5-POSITION, UNSHIELDED, ...",KUMARI MONIKA (KUMARIMONIKA),0,18-OCT-2022 11:35,4.0,215.0,0.0,0.0,...,0.0,PHEONIX.C,1424689,PXC.1424689,NaN,NaN,PRODUCTION,13-JUL-2024 21:11,ELECTRICAL PART,14858
4,14859,"AIL ADAPTER FOR M3 SCREWS,,BLACK","WIDTH10 MM, HEIGHT19 MM, LENGTH42.6 MM",KUMARI MONIKA (KUMARIMONIKA),0,18-OCT-2022 11:35,17.0,1.0,0.0,0.0,...,0.0,PHEONIX.C,1200993,PXC.1200993,NaN,NaN,NaN,NaN,ELECTRICAL PART,14859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9346,65209,"PRE FAB CABLE-3M, WIRE HARNESS, ECONOSEAL, FRE...",PRE FABRICATED AUTOMOTIVE CONNECTORS WITH FLYI...,KUMARI MONIKA (KUMARIMONIKA),0,25-SEP-2024 17:40,29.0,1.0,0.0,0.0,...,0.0,MOTHERSON,AT0699AA,MSSL.AT0699AA,NaN,NaN,PRODUCTION,26-SEP-2024 10:15,ELECTRICAL PART,65209
9347,65210,CONNECTOR- RJ45,RJ45 90 DEGREE CONNECTOR,KUMARI MONIKA (KUMARIMONIKA),0,25-SEP-2024 17:40,17.0,1.0,0.0,0.0,...,0.0,PHOENIX.C,1421128,PXC.CUC-IND - C1ZNI - T/R4IP8,NaN,NaN,PRODUCTION,26-SEP-2024 10:15,ELECTRICAL PART,65210
9348,65220,"LUG-CU, TUBE, 10MM2",CABLE LUG; SUITABLE CRIMPING INSERT: B7; FOR C...,KUMARI MONIKA (KUMARIMONIKA),0,25-SEP-2024 17:54,17.0,1.0,10.0,29.0,...,0.0,LAPP,61796650,LAPP.61796650,NaN,NaN,PRODUCTION,26-SEP-2024 10:15,ELECTRICAL PART,65220
9349,65221,"COMM MOD-WIFI,CLIENT, TRX A/B, PCI EXPRESS ,20M",WIFI5 11AC 2TX2R + BT (V5.0 LE) COMBO MODULE W...,KUMARI MONIKA (KUMARIMONIKA),0,25-SEP-2024 17:56,8.0,1.0,26.8,30.0,...,0.0,ENLI,Q6174AH,ENL-Q6174AH,NaN,NaN,NaN,NaN,ELECTRICAL PART,65221


### Merge the 2 cleaned datasets on Item Code & Part Id

In [19]:
merged_data_item_code = pd.merge(non_nan_filtered, electrical_cleaned_filtered, how="inner", left_on="ITEM_CODE_CLEANED", right_on="PART_ID_CLEANED")

In [20]:
merged_data_item_code.shape

(9796, 82)

### Prepare the remainder dataset for replacement

In [21]:
electric_data_uniq_hash_num = merged_data_item_code['#'].to_numpy()
len(electric_data_uniq_hash_num)

9796

In [22]:
remainder_dataset = non_nan_filtered[~non_nan_filtered['#'].isin(electric_data_uniq_hash_num)]
remainder_dataset

,#,DOCTYPE,CANCELED,DOCCUR,DOCRATE,DOCSTATUS,PO_NUM,BASE_TYPE,BASE_NUMBER,BASE_LINE,...,PO_OWNER_NAME,PYMNTGROUP,HEADER,FOOTER,BRANCH_NAME,IMP_OR_EXP,COST_SAVING,COST_SAVING_CRITERIA,FORCE_CLOSED_PO_QTY,ITEM_CODE_CLEANED
0,4,I,N,INR,1.0,C,51038,-1,NaN,NaN,...,NaN,NET 15 DAYS,NaN,NaN,NOIDA,N,NaN,NaN,140000.0,ELC10849
1,5,I,N,INR,1.0,C,51038,-1,NaN,NaN,...,NaN,NET 15 DAYS,NaN,NaN,NOIDA,N,NaN,NaN,100000.0,ELC10884
3,10,I,Y,INR,1.0,C,53586,-1,NaN,NaN,...,NaN,NET 15 DAYS,PLEASE SEE THE BELOW TENTATIVE DELIVERY TIMELI...,NaN,NOIDA,N,NaN,NaN,0.0,EL14328
4,12,I,N,INR,1.0,C,53650,-1,NaN,NaN,...,NaN,100 % AGAINST MATERIAL DELIVERY,NaN,NaN,NOIDA,N,NaN,NaN,1.0,EL14297
5,14,I,N,INR,1.0,C,53684,-1,NaN,NaN,...,NaN,100 % AGAINST MATERIAL DELIVERY,NaN,NaN,NOIDA,N,NaN,NaN,1.0,GC00337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42154,50454,I,N,INR,1.0,O,242530645,PR,242550434.0,1.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,16132
42155,50455,I,N,INR,1.0,O,242530645,PR,242550443.0,20.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,16135
42156,50458,I,N,INR,1.0,O,242530645,PR,242550443.0,4.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,175
42157,50460,I,N,INR,1.0,O,242530645,PR,242550420.0,2.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,22952


In [23]:
# No duplicates for the combination of PO_NUM and ITEM_CODE_CLEANED found
remainder_dataset[remainder_dataset.duplicated(subset=['PO_NUM', 'ITEM_CODE_CLEANED'], keep=False)]

,#,DOCTYPE,CANCELED,DOCCUR,DOCRATE,DOCSTATUS,PO_NUM,BASE_TYPE,BASE_NUMBER,BASE_LINE,...,PO_OWNER_NAME,PYMNTGROUP,HEADER,FOOTER,BRANCH_NAME,IMP_OR_EXP,COST_SAVING,COST_SAVING_CRITERIA,FORCE_CLOSED_PO_QTY,ITEM_CODE_CLEANED


In [24]:
remainder_dataset['ITEM_CODE_CLEANED'].value_counts()

ITEM_CODE_CLEANED
EL14568    37
GC00166    36
14437      33
EL14570    25
EL14566    23
           ..
2415601     1
16226       1
16442       1
HW19117     1
56228       1
Name: count, Length: 17256, dtype: int64

In [25]:
remainder_dataset[['PO_NUM', 'SUPPLIER_CODE', 'ITEM_CODE_CLEANED']].nunique()

PO_NUM                9434
SUPPLIER_CODE         1043
ITEM_CODE_CLEANED    17256
dtype: int64

### Separate the Remainder dataset by ITEM_CODE starting with letter vs number

In [54]:
def is_item_code_with_letter(value):
    pattern = r'^[A-Za-z]{2,}\d{5,}$'
    match = re.search(pattern, value)

    if match:
        return True

    return False

In [ ]:
# remainder_dataset_letter_based = remainder_dataset[remainder_dataset['ITEM_CODE_CLEANED'].apply(is_item_code_with_letter)]
# remainder_dataset_number_based = remainder_dataset[~remainder_dataset['#'].isin(remainder_dataset_letter_based['#'])]

### Replace the ITEM_CODE for PART_ID of electrical part based on selection criteria

In [26]:
def find_replacement(replacement_row, electrical_df, tolerance_price=0.3, tolerance_quantity=0.3):
    price_range = (replacement_row['PRICE'] * (1 - tolerance_price), replacement_row['PRICE'] * (1 + tolerance_price))
    quantity_range = (replacement_row['ORDERED_QUANTITY'] * (1 - tolerance_quantity), replacement_row['ORDERED_QUANTITY'] * (1 + tolerance_quantity))

    # Filter electrical parts that fall within the price and quantity range
    matching_parts = electrical_df[
        (electrical_df['PRICE'] >= price_range[0]) & (electrical_df['PRICE'] <= price_range[1]) &
        (electrical_df['ORDERED_QUANTITY'] >= quantity_range[0]) & (electrical_df['ORDERED_QUANTITY'] <= quantity_range[1])
    ]

    # If there are matching parts, return a random one
    if not matching_parts.empty:
        return matching_parts.sample(1).iloc[0]
    return None

In [27]:
remainder_dataset_modified = remainder_dataset.copy()

In [28]:
for index, row in remainder_dataset_modified.iterrows():
    replacement = find_replacement(row, merged_data_item_code)
    if replacement is not None:
        remainder_dataset_modified.loc[index, 'ITEM_CODE_CLEANED'] = replacement['ITEM_CODE_CLEANED']
        remainder_dataset_modified.loc[index, 'ITEM_NAME'] = replacement['ITEM_NAME']
        remainder_dataset_modified.loc[index, 'FG_ITEM_DESCRIPTION'] = replacement['FG_ITEM_DESCRIPTION']
        remainder_dataset_modified.loc[index, 'ITEM_DETAILS'] = replacement['ITEM_DETAILS']
        remainder_dataset_modified.loc[index, 'HSN'] = replacement['HSN']
        remainder_dataset_modified.loc[index, 'UOM'] = replacement['UOM']
        
remainder_dataset_modified

,#,DOCTYPE,CANCELED,DOCCUR,DOCRATE,DOCSTATUS,PO_NUM,BASE_TYPE,BASE_NUMBER,BASE_LINE,...,PO_OWNER_NAME,PYMNTGROUP,HEADER,FOOTER,BRANCH_NAME,IMP_OR_EXP,COST_SAVING,COST_SAVING_CRITERIA,FORCE_CLOSED_PO_QTY,ITEM_CODE_CLEANED
0,4,I,N,INR,1.0,C,51038,-1,NaN,NaN,...,NaN,NET 15 DAYS,NaN,NaN,NOIDA,N,NaN,NaN,140000.0,ELC10849
1,5,I,N,INR,1.0,C,51038,-1,NaN,NaN,...,NaN,NET 15 DAYS,NaN,NaN,NOIDA,N,NaN,NaN,100000.0,ELC10884
3,10,I,Y,INR,1.0,C,53586,-1,NaN,NaN,...,NaN,NET 15 DAYS,PLEASE SEE THE BELOW TENTATIVE DELIVERY TIMELI...,NaN,NOIDA,N,NaN,NaN,0.0,17685
4,12,I,N,INR,1.0,C,53650,-1,NaN,NaN,...,NaN,100 % AGAINST MATERIAL DELIVERY,NaN,NaN,NOIDA,N,NaN,NaN,1.0,29004
5,14,I,N,INR,1.0,C,53684,-1,NaN,NaN,...,NaN,100 % AGAINST MATERIAL DELIVERY,NaN,NaN,NOIDA,N,NaN,NaN,1.0,17413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42154,50454,I,N,INR,1.0,O,242530645,PR,242550434.0,1.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,26303
42155,50455,I,N,INR,1.0,O,242530645,PR,242550443.0,20.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,40789
42156,50458,I,N,INR,1.0,O,242530645,PR,242550443.0,4.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,28873
42157,50460,I,N,INR,1.0,O,242530645,PR,242550420.0,2.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,25399


In [29]:
remainder_dataset_cleaned = remainder_dataset_modified[remainder_dataset_modified['ITEM_CODE_CLEANED'].isin(electrical_cleaned_filtered['PART_ID_CLEANED'])]
remainder_dataset_cleaned

,#,DOCTYPE,CANCELED,DOCCUR,DOCRATE,DOCSTATUS,PO_NUM,BASE_TYPE,BASE_NUMBER,BASE_LINE,...,PO_OWNER_NAME,PYMNTGROUP,HEADER,FOOTER,BRANCH_NAME,IMP_OR_EXP,COST_SAVING,COST_SAVING_CRITERIA,FORCE_CLOSED_PO_QTY,ITEM_CODE_CLEANED
3,10,I,Y,INR,1.0,C,53586,-1,NaN,NaN,...,NaN,NET 15 DAYS,PLEASE SEE THE BELOW TENTATIVE DELIVERY TIMELI...,NaN,NOIDA,N,NaN,NaN,0.0,17685
4,12,I,N,INR,1.0,C,53650,-1,NaN,NaN,...,NaN,100 % AGAINST MATERIAL DELIVERY,NaN,NaN,NOIDA,N,NaN,NaN,1.0,29004
5,14,I,N,INR,1.0,C,53684,-1,NaN,NaN,...,NaN,100 % AGAINST MATERIAL DELIVERY,NaN,NaN,NOIDA,N,NaN,NaN,1.0,17413
6,15,I,Y,INR,1.0,C,53946,-1,NaN,NaN,...,NaN,100% ADVANCE,NaN,NaN,NOIDA,N,NaN,NaN,0.0,18889
7,16,I,N,INR,1.0,C,53969,-1,NaN,NaN,...,NaN,REFER PO TEXT,NaN,NaN,NOIDA,N,NaN,NaN,1.0,42992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42154,50454,I,N,INR,1.0,O,242530645,PR,242550434.0,1.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,26303
42155,50455,I,N,INR,1.0,O,242530645,PR,242550443.0,20.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,40789
42156,50458,I,N,INR,1.0,O,242530645,PR,242550443.0,4.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,28873
42157,50460,I,N,INR,1.0,O,242530645,PR,242550420.0,2.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,25399


In [30]:
po_electrical_data_cleaned = pd.concat([df_cleaned_non_nan, remainder_dataset_cleaned])
po_electrical_data_cleaned_mod = po_electrical_data_cleaned.groupby(['PO_NUM', 'ITEM_CODE_CLEANED']).apply(keep_most_quantity).reset_index(drop=True)
po_electrical_data_cleaned_mod

/var/folders/4c/99wh1f85111309s8ydsfjmlr0000gn/T/ipykernel_49860/2531319751.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  po_electrical_data_cleaned_mod = po_electrical_data_cleaned.groupby(['PO_NUM', 'ITEM_CODE_CLEANED']).apply(keep_most_quantity).reset_index(drop=True)


,#,DOCTYPE,CANCELED,DOCCUR,DOCRATE,DOCSTATUS,PO_NUM,BASE_TYPE,BASE_NUMBER,BASE_LINE,...,PO_OWNER_NAME,PYMNTGROUP,HEADER,FOOTER,BRANCH_NAME,IMP_OR_EXP,COST_SAVING,COST_SAVING_CRITERIA,FORCE_CLOSED_PO_QTY,ITEM_CODE_CLEANED
0,4,I,N,INR,1.0,C,51038,-1,NaN,NaN,...,NaN,NET 15 DAYS,NaN,NaN,NOIDA,N,NaN,NaN,140000.0,ELC10849
1,5,I,N,INR,1.0,C,51038,-1,NaN,NaN,...,NaN,NET 15 DAYS,NaN,NaN,NOIDA,N,NaN,NaN,100000.0,ELC10884
2,9,I,Y,INR,1.0,C,53586,-1,NaN,NaN,...,NaN,NET 15 DAYS,PLEASE SEE THE BELOW TENTATIVE DELIVERY TIMELI...,NaN,NOIDA,N,NaN,NaN,0.0,17340
3,10,I,Y,INR,1.0,C,53586,-1,NaN,NaN,...,NaN,NET 15 DAYS,PLEASE SEE THE BELOW TENTATIVE DELIVERY TIMELI...,NaN,NOIDA,N,NaN,NaN,0.0,17685
4,10,I,Y,INR,1.0,C,53586,-1,NaN,NaN,...,NaN,NET 15 DAYS,PLEASE SEE THE BELOW TENTATIVE DELIVERY TIMELI...,NaN,NOIDA,N,NaN,NaN,0.0,EL14328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71325,50458,I,N,INR,1.0,O,242530645,PR,242550443.0,4.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,28873
71326,50461,I,N,INR,1.0,O,242530645,PR,242550438.0,36.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,36349
71327,50455,I,N,INR,1.0,O,242530645,PR,242550443.0,20.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,40789
71328,50459,I,N,INR,1.0,O,242530645,PR,242550443.0,13.0,...,POOJA YADAV,NET-30,NaN,NaN,ECOTECH X,N,NaN,NaN,0.0,43015


In [31]:
merged_data_item_code_2 = pd.merge(po_electrical_data_cleaned_mod, electrical_cleaned_filtered, how="inner", left_on="ITEM_CODE_CLEANED", right_on="PART_ID_CLEANED")
merged_data_item_code_2

,#,DOCTYPE,CANCELED,DOCCUR,DOCRATE,DOCSTATUS,PO_NUM,BASE_TYPE,BASE_NUMBER,BASE_LINE,...,WEIGHT_(KG),MANUFACTURER,MANUFACTURER_PART_NUMBER,EPLAN_PART_NUMBER,OLD_PLM_ID,OLD_SAP_ID,RELEASED_STATUS,RELEASED_DATE,PART_CATEGORY,PART_ID_CLEANED
0,9,I,Y,INR,1.0,C,53586,-1,NaN,NaN,...,0.000,AAEON.T,UPS-APLC2F-A20-0432,AON.UPS-APLC2F-A20-0432,NaN,NaN,PRODUCTION,13-JUL-2023 15:41,ELECTRICAL PART,17340
1,10,I,Y,INR,1.0,C,53586,-1,NaN,NaN,...,0.027,MURRELE,7000-P4611-0000000,MURR.7000-P4611-0000000,NaN,NaN,PRODUCTION,04-AUG-2024 19:35,ELECTRICAL PART,17685
2,12,I,N,INR,1.0,C,53650,-1,NaN,NaN,...,0.000,SIEMENS,6SL3055-0AA00-4BA0,SIE.6SL3055-0AA00-4BA0,NaN,NaN,PRODUCTION,11-JAN-2023 16:55,ELECTRICAL PART,29004
3,14,I,N,INR,1.0,C,53684,-1,NaN,NaN,...,0.000,SCHNEIDER,LC1D09BL,SE.LC1D09BL,NaN,NaN,NaN,NaN,ELECTRICAL PART,17413
4,15,I,Y,INR,1.0,C,53946,-1,NaN,NaN,...,0.200,P+F,293431-100004,P+F.293431-100004,NaN,NaN,NaN,NaN,ELECTRICAL PART,18889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38962,50454,I,N,INR,1.0,O,242530645,PR,242550434.0,1.0,...,NaN,MOLEX,430300002,MOL.430300002,NaN,NaN,PRODUCTION,12-DEC-2022 18:36,ELECTRICAL PART,26303
38963,50458,I,N,INR,1.0,O,242530645,PR,242550443.0,4.0,...,NaN,LAPP,4510021U,LAPP.4510021U,NaN,NaN,PRODUCTION,13-JAN-2023 19:43,ELECTRICAL PART,28873
38964,50455,I,N,INR,1.0,O,242530645,PR,242550443.0,20.0,...,NaN,LAPP,61801930,LAPP.61801930,NaN,NaN,PRODUCTION,25-AUG-2023 14:39,ELECTRICAL PART,40789
38965,50459,I,N,INR,1.0,O,242530645,PR,242550443.0,13.0,...,0.000,CWT,EP6/10U,CWT.EP6/10U,NaN,NaN,PRODUCTION,30-SEP-2023 16:29,ELECTRICAL PART,43015


In [32]:
merged_data_item_code_2.to_csv('filtered_data_final_2.csv')

### Replace the ITEM_CODE with NaN value for PART_ID of electrical part based on selection criteria

In [33]:
df_cleaned_nan = df_cleaned[df_cleaned['ITEM_CODE'].isna()].copy()
df_cleaned_nan

,#,DOCTYPE,CANCELED,DOCCUR,DOCRATE,DOCSTATUS,PO_NUM,BASE_TYPE,BASE_NUMBER,BASE_LINE,...,MANUALLY_CLOSED_PO,PO_OWNER_NAME,PYMNTGROUP,HEADER,FOOTER,BRANCH_NAME,IMP_OR_EXP,COST_SAVING,COST_SAVING_CRITERIA,FORCE_CLOSED_PO_QTY
2,45582,S,N,INR,1.0,C,232472169,PR,232452086.0,1.0,...,NO,ABHISHEK CHAUHAN,REFER PO TEXT,PAYMENT TERMS : 100% AFTER SERVICE COMPLETION ...,NaN,ECOTECH X,N,NaN,NaN,0.0
3,9868,S,N,INR,1.0,C,72616,PR,100150.0,1.0,...,YES,AMAN CHAMBIAL,NET-30,OTC CHARGES SHALL BE PAID AFTER INSTALLATION O...,NaN,NOIDA,N,NaN,NaN,0.0
4,9869,S,N,INR,1.0,C,72616,PR,100150.0,2.0,...,YES,AMAN CHAMBIAL,NET-30,OTC CHARGES SHALL BE PAID AFTER INSTALLATION O...,NaN,NOIDA,N,NaN,NaN,0.0
5,9870,S,N,INR,1.0,C,72616,PR,100150.0,3.0,...,NO,AMAN CHAMBIAL,NET-30,OTC CHARGES SHALL BE PAID AFTER INSTALLATION O...,NaN,NOIDA,N,NaN,NaN,0.0
6,22886,S,N,INR,1.0,C,232420215,PR,232404753.0,1.0,...,NO,NEERAJ TIWARI,NET-30,NaN,NaN,NOIDA,N,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50271,37350,S,N,INR,1.0,C,232450057,PR,232440055.0,4.0,...,NO,DHEERAJ SACHDEVA,NET 30 DAYS,IF ZOOM CONTACT CENTER CLOUD STORAGE 200GB REQ...,NaN,AFSNOIDA,N,NaN,NaN,0.0
50272,37974,S,N,INR,1.0,O,232450263,PR,232440259.0,1.0,...,NO,SANDEEP KUMAR GAUTAM,NET 30 DAYS,NaN,NaN,AFSNOIDA,N,NaN,NaN,0.0
50273,37975,S,N,INR,1.0,O,232450263,PR,232440259.0,2.0,...,NO,SANDEEP KUMAR GAUTAM,NET 30 DAYS,NaN,NaN,AFSNOIDA,N,NaN,NaN,0.0
50274,37976,S,N,INR,1.0,O,232450263,PR,232440259.0,3.0,...,NO,SANDEEP KUMAR GAUTAM,NET 30 DAYS,NaN,NaN,AFSNOIDA,N,NaN,NaN,0.0


In [43]:
df_cleaned_nan[['PO_NUM', 'SUPPLIER_CODE']].nunique()

PO_NUM           2462
SUPPLIER_CODE     692
dtype: int64

In [44]:
df_cleaned_nan.to_csv('data_with_nan_item_code.csv')

In [41]:
electric_parts_not_used = electrical_cleaned_filtered[~electrical_cleaned_filtered['PART_ID_CLEANED'].isin(merged_data_item_code_2['ITEM_CODE_CLEANED'])]
electric_parts_not_used = electric_parts_not_used[electric_parts_not_used['PART_ID_CLEANED'] != electric_parts_not_used['PART_NAME']]

In [42]:
electric_parts_not_used

,PART_ID,PART_NAME,PART_DESCRIPTION,PART_OWNER,PART_REVISION,REVISION_DATE,PRODUCT_GROUP,PRODUCT_SUBGROUP,WIDTH_(MM),HEIGHT_(MM),...,WEIGHT_(KG),MANUFACTURER,MANUFACTURER_PART_NUMBER,EPLAN_PART_NUMBER,OLD_PLM_ID,OLD_SAP_ID,RELEASED_STATUS,RELEASED_DATE,PART_CATEGORY,PART_ID_CLEANED
2,14857,"CABLE,M8 RIGHT ANGLED TO FREE LEADS,2M","SENSOR/ACTUATOR CABLE, 3-POSITION, PVC, YELLOW...",KUMARI MONIKA (KUMARIMONIKA),0,18-OCT-2022 11:35,29.0,184.0,0.0,0.0,...,0.0,PHEONIX.C,1406321,PXC.1406321,NaN,NaN,NaN,NaN,ELECTRICAL PART,14857
4,14859,"AIL ADAPTER FOR M3 SCREWS,,BLACK","WIDTH10 MM, HEIGHT19 MM, LENGTH42.6 MM",KUMARI MONIKA (KUMARIMONIKA),0,18-OCT-2022 11:35,17.0,1.0,0.0,0.0,...,0.0,PHEONIX.C,1200993,PXC.1200993,NaN,NaN,NaN,NaN,ELECTRICAL PART,14859
5,14860,DISTRIBUTION BLOCK,NaN,KUMARI MONIKA (KUMARIMONIKA),0,18-OCT-2022 11:35,3.0,4.0,0.0,0.0,...,0.0,PHEONIX.C,3273224,PXC.3273224,NaN,NaN,PRODUCTION,21-SEP-2024 11:01,ELECTRICAL PART,14860
6,14861,DISTRIBUTION BLOCK,NaN,KUMARI MONIKA (KUMARIMONIKA),0,18-OCT-2022 11:35,3.0,4.0,0.0,0.0,...,0.0,PHEONIX.C,3273234,PXC.3273234,NaN,NaN,PRODUCTION,21-SEP-2024 11:01,ELECTRICAL PART,14861
7,14862,DISTRIBUTION BLOCK,"PTFIX 6/12X2,5-NS35A YE 3273226 PHOENIX CONTAC...",KUMARI MONIKA (KUMARIMONIKA),0,18-OCT-2022 11:35,3.0,4.0,0.0,0.0,...,0.0,PHEONIX.C,3273226,PXC.3273226,NaN,NaN,NaN,NaN,ELECTRICAL PART,14862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9346,65209,"PRE FAB CABLE-3M, WIRE HARNESS, ECONOSEAL, FRE...",PRE FABRICATED AUTOMOTIVE CONNECTORS WITH FLYI...,KUMARI MONIKA (KUMARIMONIKA),0,25-SEP-2024 17:40,29.0,1.0,0.0,0.0,...,0.0,MOTHERSON,AT0699AA,MSSL.AT0699AA,NaN,NaN,PRODUCTION,26-SEP-2024 10:15,ELECTRICAL PART,65209
9347,65210,CONNECTOR- RJ45,RJ45 90 DEGREE CONNECTOR,KUMARI MONIKA (KUMARIMONIKA),0,25-SEP-2024 17:40,17.0,1.0,0.0,0.0,...,0.0,PHOENIX.C,1421128,PXC.CUC-IND - C1ZNI - T/R4IP8,NaN,NaN,PRODUCTION,26-SEP-2024 10:15,ELECTRICAL PART,65210
9348,65220,"LUG-CU, TUBE, 10MM2",CABLE LUG; SUITABLE CRIMPING INSERT: B7; FOR C...,KUMARI MONIKA (KUMARIMONIKA),0,25-SEP-2024 17:54,17.0,1.0,10.0,29.0,...,0.0,LAPP,61796650,LAPP.61796650,NaN,NaN,PRODUCTION,26-SEP-2024 10:15,ELECTRICAL PART,65220
9349,65221,"COMM MOD-WIFI,CLIENT, TRX A/B, PCI EXPRESS ,20M",WIFI5 11AC 2TX2R + BT (V5.0 LE) COMBO MODULE W...,KUMARI MONIKA (KUMARIMONIKA),0,25-SEP-2024 17:56,8.0,1.0,26.8,30.0,...,0.0,ENLI,Q6174AH,ENL-Q6174AH,NaN,NaN,NaN,NaN,ELECTRICAL PART,65221


In [45]:
electric_parts_not_used.to_csv('electrical_parts_not_used.csv')